In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [9]:
class Discriminator(nn.Module):
    def __init__(self,channels,features_d):
        super(Discriminator,self).__init__()
        self.disc=nn.Sequential(
            nn.Conv2d(channels,features_d,4,2,1),
            nn.LeakyReLU(0.2),
            self._block(features_d,features_d*2,4,2,1),
            self._block(features_d*2,features_d*4,4,2,1),
            self._block(features_d*4,features_d*8,4,2,1),
            nn.Conv2d(features_d*8,1,4,2,0),
            nn.Sigmoid()
        )

    def _block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    def forward(self,x):
        return self.disc(x)

In [10]:
class Generator(nn.Module):
    def _block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    def __init__(self,z_dim,img_channels,features_g):
        super(Generator,self).__init__()
        self.gen=nn.Sequential(
            self._block(z_dim,features_g*8,4,1,0),
            self._block(features_g*8,features_g*4,4,2,1),
            self._block(features_g*4,features_g*2,4,2,1),
            self._block(features_g*2,features_g*1,4,2,1),
            nn.ConvTranspose2d(features_g,img_channels,4,2,1),
            nn.Tanh()
        )
    def forward(self,x):
        return self.gen(x)

In [11]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m,(nn.ConvTranspose2d,nn.Conv2d,nn.BatchNorm2d)):
            nn.init.normal_(m.weight,0.0,0.02)

In [12]:
def test():
    N,in_channels,H,W=8,3,64,64
    z_dim=100
    x=torch.randn((N,in_channels,H,W))
    disc=Discriminator(in_channels,8)
    initialize_weights(disc)
    assert disc(x).shape==(N,1,1,1 )
    gen=Generator(z_dim,in_channels,8)
    initialize_weights(gen)
    z=torch.randn((N,z_dim,1,1))
    assert gen(z).shape==(N,in_channels,H,W)
    print("Success")
test()

Success


In [13]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [14]:
lr=2e-4
lr

0.0002

In [15]:
img_size=64
batch_size=128
z_dim=256
img_channels=3
features_d=64
features_g=64

In [16]:
transform=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(img_channels)],[0.5 for _ in range(img_channels)])
])

In [18]:
dataset=datasets.CelebA(root='data/',download=True,transform=transform)

FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM

but Gdown can't. Please check connections and permissions.